In [1]:
# Requires:
# pip install langchain docarray tiktoken

from langchain.llms import LlamaCpp
llm = LlamaCpp(model_path="/home/tpllmws23/llms/llama-2-13b-chat.Q4_K_M.gguf",
        n_gpu_layers=-1,
        n_batch=512,
        n_ctx=2048,
        f16_kv=True,
        verbose=False,
        temperature=0.0,
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA TITAN RTX, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/tpllmws23/llms/llama-2-13b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length

In [3]:
%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, PyPDFDirectoryLoader, PyPDFLoader
from langchain_community.embeddings import LlamaCppEmbeddings
OpenAIEmbeddings
#Use Llama model for embedding
llama_model_path = "/home/tpllmws23/llms/llama-2-13b-chat.Q4_K_M.gguf"
embeddings = LlamaCppEmbeddings()

# Load the document, split it into chunks, embed each chunk and load it into the vector store.

raw_documents = PyPDFDirectoryLoader("./main_data/").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)



llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/tpllmws23/llms/llama-2-13b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_cou

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [5]:

db = Chroma.from_documents(documents, embeddings)

: 

In [3]:

from langchain.embeddings import LlamaCppEmbeddings

#Use Llama model for embedding
llama_model_path = "/home/tpllmws23/llms/llama-2-13b-chat.Q4_K_M.gguf"
embeddings = LlamaCppEmbeddings(model_path=llama_model_path, verbose=False)


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/tpllmws23/llms/llama-2-13b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_cou

In [4]:
# load chromadb as retriever

from operator import itemgetter
from langchain_community.vectorstores.chroma import Chroma
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings.llamacpp import LlamaCppEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFDirectoryLoader, PyPDFLoader
import chromadb
import os
from langchain.text_splitter import CharacterTextSplitter

if not os.path.exists("./chroma_db2"): 
    docs = PyPDFDirectoryLoader("./main_data").load_and_split(text_splitter=CharacterTextSplitter())

    db = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="./chroma_db2", collection_name="pruefungsamt")

    retriever = db.as_retriever()
else:
    db = Chroma(persist_directory="./chroma_db", collection_name="pruefungsamt", embedding_function=embeddings)
    retriever = db.as_retriever()


: 

In [ ]:

    db.persist()

In [9]:
# split it into chunks

In [5]:
db.similarity_search("Anforderung Informatik Master")

InvalidDimensionException: Embedding dimension 5120 does not match collection dimensionality 384

In [6]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | db.as_retriever(),
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
chain.invoke({"question": "What are the requirements for studying MSI in masters degree at the HTWG?", "language": "german"})

InvalidDimensionException: Embedding dimension 5120 does not match collection dimensionality 384